In [1]:
import pandas as pd

data = pd.read_csv('source.csv')
data.drop_duplicates(subset=['char'], keep='first', inplace=True)
data.sort_values(by=['char'], inplace=True)

C:\Users\liuba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


第一步係要將唔需要嘅詞條都清走擺埋一邊，呢啲詞條包括：

1. 字數大於 4 嘅詞條
1. 唔常用嘅專名，包括：
   1. 街道名路名
   1. 城市村寨樓盤名
   1. 車站名
   1. 商鋪名
   1. 機構組織名
1. 幾個詞表入面已經有嘅名


In [2]:
pattern = r'(店|路|道|巷|莊|鎮|苑|市|街|村|站|綫|學|校|樓|場|館|社|屋|岭|所|行|園|城|中心|百貨|大廈|屋|房|廳|廠|院|酒吧|網吧|印刷|教育|文具|化工|社區|實業|酒家|科技|餐|廊|公司|女裝|時裝|男裝|電訊)$'

proper_nouns = data[data['char'].str.contains(pattern)]
remain = data[~data['char'].str.contains(pattern)]

long = remain[remain['char'].str.len() > 4]
remain = remain[remain['char'].str.len() <= 4]

C:\Users\liuba\AppData\Local\Temp/ipykernel_27128/179914394.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  proper_nouns = data[data['char'].str.contains(pattern)]
C:\Users\liuba\AppData\Local\Temp/ipykernel_27128/179914394.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  remain = data[~data['char'].str.contains(pattern)]


然後要將已經有嘅嗰啲數據都清晒佢，包括冚唪唥詞表粵典詞表 CyberCan 詞表等等。

In [3]:
hbl = pd.read_csv('hbl.csv')
condition = remain['char'].isin(hbl['char'])
remain.drop(remain[condition].index, inplace = True)

wordshk = pd.read_csv('wordshk.csv')
condition = remain['char'].isin(wordshk['char'])
remain.drop(remain[condition].index, inplace = True)

cybercan = pd.read_csv('CyberCan.csv')
condition = remain['char'].isin(cybercan['Words'])
remain.drop(remain[condition].index, inplace = True)

# 下面嘅都係ngram，唔可以當詞嚟用
apple_daily = pd.read_csv('AppleDaily.csv')
condition = remain['char'].isin(apple_daily['ngram'])
remain.drop(remain[condition].index, inplace = True)

wiki = pd.read_csv('wiki.csv')
condition = remain['char'].isin(wiki['ngram'])
remain.drop(remain[condition].index, inplace = True)


跟住要將啲官話詞抽出嚟

In [4]:
# common_words = pd.concat([hbl, wordshk, cybercan])
# common_words.drop_duplicates(subset=['char'], keep='first', inplace=True)

# common_two_words = common_words[common_words['char'].str.len() == 2]

# combo_words = []
# for index, four_word in remain_four.iterrows():
#     if any(map(four_word['char'].__contains__,common_two_words['char'])):
#       combo_words.append(four_word)

mandarin_words = r'(這|那|吧|他|她|它|們|是|的|哪)'
mandarin = remain[remain['char'].str.contains(mandarin_words)]
remain = remain[~remain['char'].str.contains(mandarin_words)]


remain.to_csv('remain.csv', index=False)
long.to_csv('remain_long.csv', index=False)


C:\Users\liuba\AppData\Local\Temp/ipykernel_27128/2427951999.py:12: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  mandarin = remain[remain['char'].str.contains(mandarin_words)]
C:\Users\liuba\AppData\Local\Temp/ipykernel_27128/2427951999.py:13: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  remain = remain[~remain['char'].str.contains(mandarin_words)]


然後打亂佢，分成一行一千個詞導出到個文件夾入面

In [11]:
shuffled = remain.sample(frac=1)

num_slices = shuffled.shape[0] // 1000 + 1

for i in range(num_slices):
    start = i * 1000
    end = (i + 1) * 1000
    sliced = shuffled.iloc[start:end]
    sliced.to_csv(f'./round_1_raw/r1_{i+1}.csv', index=False)